# IMPORT LIBRARY

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load Model

In [ ]:
!gdown --id 1IeKLpYxK4M7FCW14UPLROzBMkpTxRHpi

Downloading...
From: https://drive.google.com/uc?id=1IeKLpYxK4M7FCW14UPLROzBMkpTxRHpi
To: /content/model.h5
100% 37.5k/37.5k [00:00<00:00, 9.50MB/s]


In [ ]:
model = load_model('model.h5')

# Rescaling value

In [ ]:
fullnamewords_maxmin = [12, 5]
fullnamelen_maxmin = [50,0]
desclen_maxmin = [150, 0]
posts_maxmin = [299 * 3, 0]
followers_maxmin = [110*3, 0]
following_maxmin = [302*3, 0]

In [ ]:
def rescale_uname(value):
    max = 15
    min = 4
    if value > max:
        value = 1.0
    elif value < min:
        value = 0.0
    else:
        value = (value-min)/(max-min)
    return value

def rescale(value, maxmin):
    maxval = maxmin[0]
    minval = maxmin[1]
    if value > maxval:
        value = 1.0
    elif value < minval:
        value = 0.0
    else:
        value = (value-minval)/(maxval-minval)
    return value

# Method Preprocessing Data Sebelum dimasukan Ke Model

In [ ]:
def processing_profil(profil_pic, username, fullname, desc, private, posts, followers, following):
#     username = username.replace('@', '')
    username = username.lower()
#     print("username = {}".format(username))
    username_len = rescale_uname(len(username))
#     print("username length = {}".format(username_len))
    fullname_words = fullname.split()
    fullname_words = len(fullname_words)
    fullname_words = rescale(fullname_words, fullnamewords_maxmin)
#     print("Fullname words = {}".format(fullname_words))
    fullname_len = rescale(len(fullname), fullnamelen_maxmin)
#     print("Fullname length = {}".format(fullname_len))
    fullname = fullname.replace(' ', '')
    fullname = fullname.lower()
    
    if fullname == username:
        same = 1
    else:
        same = 0
#     print("is same fullname username = {}".format(same))
    
    desc_len = rescale(len(desc), desclen_maxmin)
#     print("Desc length = {}".format(desc_len))
    num_post = rescale(posts, posts_maxmin)
#     print("post = {}".format(num_post))
    followers = rescale(followers, followers_maxmin)
#     print("followers = {}".format(followers))
    following = rescale(following, following_maxmin)
#     print("following = {}".format(following))
    if profil_pic is None or profil_pic is 'null':
        profil_pic = 0
    else:
        profil_pic = 1
        
    if private:
        private = 1
    else: 
        private = 0
    
    features = [
        profil_pic,
        username_len,
        fullname_words,
        fullname_len,
        same,
        desc_len,
        private,
        num_post,
        followers,
        following
    ]
    
    return features

# Customisasi Hasil Keluaran

In [ ]:
def process_result(result):
    result_list = []
    
    for res in result:
        res[0] *= 100
        if(res[0] > 50):
            result_list.append("User Probably {:.2f}% is BUZZER/FAKE Account".format(res[0]))
        else:
            result_list.append("User Probably {:.2f}% is ACTIVE Account".format(100-res[0]))
    
    return result_list

# Handler Data dari Dictionary

In [ ]:
def dict_handler(dictlist, model):
    profil_name = []
    query = []
    for dict in dictlist:
        single_list = []
        profil_pic = dict['profil_pic']
        username = dict['username']
        fullname = dict['fullname']
        desc = dict['desc']
        private = dict['private']
        posts = dict['posts']
        followers = dict['followers']
        following = dict['following']
        single_list = processing_profil(profil_pic, username, fullname, desc, private, posts, followers, following)
        profil_name.append(username)
        query.append(single_list)
    result = model.predict(query)
    result = process_result(result)
    return profil_name, result

# Function Menampilkan Hasil

In [ ]:
def show_result(owner_list, result_list):
    for i in range(len(owner_list)):
        print(owner_list[i])
        print(result_list[i])
        print()

# Silahkan Input Data Account Dalam Dictionary Ini

In [ ]:
data_dict = [
    {'profil_pic' : 'http/1231232114214',
    'username' : '@malfrzt',
    'fullname' : 'M Alfarizi',
    'desc' : 'ig : muhammadalfarizi.t',
    'private' : False,
    'posts' : 613,
    'followers' : 111,
    'following' : 89},
             
     {'profil_pic' : None,
    'username' : '@best_in_dumbest',
    'fullname' : 'The Best In Dumbest',
    'desc' : 'Blame @xaiax, Inspired by @MakingInvisible, using cmu phonetic data to produce incongruous matches.Some images via Lorem Flickr.',
    'private' : False,
    'posts' : 251,
    'followers' : 1700,
    'following' : 4},
    
    {'profil_pic' : None,
    'username' : '@CJRubinPhoto',
    'fullname' : 'CJ Rubin',
    'desc' : 'Photographing the American West since 1980. I specialize in location portraits & events, both indoors & outside, using natural light & portable studio lighting.',
    'private' : True,
    'posts' : 84100,
    'followers' : 1800000000,
    'following' : 2269},
    
    {'profil_pic' : 'httpasdadadads',
    'username' : '@JacobWhitesides',
    'fullname' : 'jacob whitesides',
    'desc' : 'Photographing the American West since 1980. I specialize in location portraits & events, both indoors & outside, using natural light & portable studio lighting.',
    'private' : False,
    'posts' : 84100,
    'followers' : 820,
    'following' : 851},
    
    {'profil_pic' : None,
    'username' : '@KaitlynTck',
    'fullname' : 'Kaitlyn • Girl Pow-R',
    'desc' : 'Girl Pow-R Trainees: The Rubies Gem stonesinger & actress 11x Scenebot Honorable Mention and 19x Top Select, Insta - kaitlyn_tck',
    'private' : True,
    'posts' : 9341,
    'followers' : 583,
    'following' : 951},
    
    {'profil_pic' : 'httpasfasfwefwe',
    'username' : '@SaputriRisty',
    'fullname' : 'Risty Saputri',
     'desc' : 'Mutualan terus',
    'private' :True,
    'posts' : 218,
    'followers' : 33,
    'following' : 49},
    ]

owner, result = dict_handler(data_dict , model)

show_result(owner, result)

@malfrzt
User Probably 81.40% is ACTIVE Account

@best_in_dumbest
User Probably 94.41% is BUZZER/FAKE Account

@CJRubinPhoto
User Probably 94.96% is ACTIVE Account

@JacobWhitesides
User Probably 92.03% is ACTIVE Account

@KaitlynTck
User Probably 97.20% is ACTIVE Account

@SaputriRisty
User Probably 99.95% is BUZZER/FAKE Account

